In [1]:
import pandas as pd
from sqlalchemy import create_engine, text
from datetime import datetime, date

In [2]:
user = 'admin'
password = 1234
host = 'localhost'
port = 5432
db_name = 'skud_va'
schema = 'fin'

engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{db_name}')
def read_sql_query(query):
    with engine.begin() as con:
        res = pd.read_sql_query(query, con)
    return res

In [3]:
table_columns = {
                 #    'revenue_fact': [ 'id::serial', 'date_dt', 
                 #                      'c_agent', 'contract', 
                 #                      'doc', 'division', 'frc',
                 #                      'nom_g', 'div_frc', 'nom_frc', 'amount'
    
                 #                    ], 
                 # 'revenue_plan_2025': ['id::serial', 'company', 
                 #                           'date_dt', 'frc', 'amount'
                 #                     ], 
                 'revenue_est_2025': ['id', 'company', 
                                          'date_dt', 'estimate_date', 
                                          'frc', 'est_amount', 
                                          'hcl_amount', 'contr_amount']}

In [4]:
def get_id_adding_query_list(table, columns, start_id):
    return [
f"create sequence fin.{table}_seq as int start with {start_id}",
f"alter table fin.{table} rename to {table}_old;",
f'alter table fin.{table}_old add column id serial;',
f"""create table fin.{table} as (
	select {', '.join(columns)} 
	from fin.{table}_old
);""",
f"alter table fin.{table} add primary key(id)",
f"ALTER TABLE fin.{table} ALTER COLUMN id SET DEFAULT nextval('fin.{table}_seq');"
f"drop table fin.{table}_old;"
]

def add_id(table_columns):
    for key, value in table_columns.items():
        start_id = read_sql_query(f'select count(*) from fin.revenue_est_2025')['count'].values[0] + 1
        id_adding_queries = get_id_adding_query_list(key, value, start_id)
        for query in id_adding_queries:
            with engine.begin() as con:
                con.execute(text(query))

add_id(table_columns)

In [19]:
def get_dfs(table_columns):
    dfs = []
    for t_name in list(table_columns.keys()):
        query = f'select * from fin.{t_name}'
        dfs.append(read_sql_query(query))
    return dfs

df_fact, df_plan, df_est = get_dfs(table_columns)

In [5]:
frc_values = df_est['frc'].value_counts().keys().to_list()

In [22]:
frc_values

['Оценка и технический контроль',
 'Центр компетенции и системы управления качеством ГК Ростех',
 'Центр сертификации',
 'Центр обучения',
 'Инновации и инжиниринг',
 'Стратегия и инвестиции',
 'Управление проектами и цифровизацией',
 'Центр качества поставок']

In [21]:
def get_est(frc):
    cur_month = datetime.now().month
    return (
        df_est
        .assign(date_dt = lambda x: x['date_dt'].astype('datetime64[ns]'),
               estimate_date = lambda x: x['estimate_date'].astype('datetime64[ns]'))
        .sort_values(['date_dt'])
        .query(f'frc == "{frc}"')
        .query(f'(estimate_date == date_dt) or ((estimate_date.dt.month == {cur_month}))')
    )

[get_est(frc).shape[0] for frc in frc_values]
# get_est(frc_values[6])

[12, 12, 12, 12, 12, 12, 12, 12]

In [9]:
month_nums = [i for i in range(1, 13)]
month_nums

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

In [10]:
adding_data_query_list = []
for frc in frc_values:
    company, frc = get_est(frc).iloc[0,:][['company', 'frc']].to_list()
    est_months = get_est(frc)['date_dt'].dt.month.to_list() 
    months_not_in_data = [m for m in month_nums if m not in est_months]
    for m in months_not_in_data:
        if m <= 8:
            date_dt = estimate_date = date(2025, m, 1).strftime("%Y-%m-%d")
        else:
            date_dt = date(2025, m, 1).strftime("%Y-%m-%d")
            estimate_date = '2025-08-01'
        adding_data_query_list.append(f"(default, '{date_dt}', '{estimate_date}', '{company}', '{frc}')")

adding_data_query_list

['(default, \'2025-04-01\', \'2025-04-01\', \'АО "РТ-Техприемка"\', \'Центр обучения\')',
 '(default, \'2025-03-01\', \'2025-03-01\', \'АО "РТ-Техприемка"\', \'Инновации и инжиниринг\')',
 '(default, \'2025-05-01\', \'2025-05-01\', \'АО "РТ-Техприемка"\', \'Инновации и инжиниринг\')',
 '(default, \'2025-01-01\', \'2025-01-01\', \'АО "РТ-Техприемка"\', \'Стратегия и инвестиции\')',
 '(default, \'2025-02-01\', \'2025-02-01\', \'АО "РТ-Техприемка"\', \'Стратегия и инвестиции\')',
 '(default, \'2025-07-01\', \'2025-07-01\', \'АО "РТ-Техприемка"\', \'Стратегия и инвестиции\')',
 '(default, \'2025-08-01\', \'2025-08-01\', \'АО "РТ-Техприемка"\', \'Стратегия и инвестиции\')',
 '(default, \'2025-09-01\', \'2025-08-01\', \'АО "РТ-Техприемка"\', \'Стратегия и инвестиции\')',
 '(default, \'2025-01-01\', \'2025-01-01\', \'АО "РТ-Техприемка"\', \'Управление проектами и цифровизацией\')',
 '(default, \'2025-02-01\', \'2025-02-01\', \'АО "РТ-Техприемка"\', \'Управление проектами и цифровизацией\')',


In [11]:
adding_data_query = rf"""insert into fin.revenue_est_2025 (id, date_dt, estimate_date, company, frc) values {", ".join(adding_data_query_list)}"""
adding_data_query

'insert into fin.revenue_est_2025 (id, date_dt, estimate_date, company, frc) values (default, \'2025-04-01\', \'2025-04-01\', \'АО "РТ-Техприемка"\', \'Центр обучения\'), (default, \'2025-03-01\', \'2025-03-01\', \'АО "РТ-Техприемка"\', \'Инновации и инжиниринг\'), (default, \'2025-05-01\', \'2025-05-01\', \'АО "РТ-Техприемка"\', \'Инновации и инжиниринг\'), (default, \'2025-01-01\', \'2025-01-01\', \'АО "РТ-Техприемка"\', \'Стратегия и инвестиции\'), (default, \'2025-02-01\', \'2025-02-01\', \'АО "РТ-Техприемка"\', \'Стратегия и инвестиции\'), (default, \'2025-07-01\', \'2025-07-01\', \'АО "РТ-Техприемка"\', \'Стратегия и инвестиции\'), (default, \'2025-08-01\', \'2025-08-01\', \'АО "РТ-Техприемка"\', \'Стратегия и инвестиции\'), (default, \'2025-09-01\', \'2025-08-01\', \'АО "РТ-Техприемка"\', \'Стратегия и инвестиции\'), (default, \'2025-01-01\', \'2025-01-01\', \'АО "РТ-Техприемка"\', \'Управление проектами и цифровизацией\'), (default, \'2025-02-01\', \'2025-02-01\', \'АО "РТ-Техп

In [12]:
with engine.begin() as con:
    con.execute(text(adding_data_query))

In [16]:
[get_est(frc).shape[0] for frc in frc_values]

[12, 12, 12, 11, 10, 7, 9, 8]

# Users

In [35]:
users_raw = pd.read_excel("../../postgres/data/users.xlsx")

In [40]:
users = pd.concat([(
    users_raw                     # Руководители структурных подразделений
    .dropna(subset=['e-mail'])
    .assign(login = lambda x: x['e-mail'].apply(lambda x: x.split('@')[0]),
            email = lambda x: x['e-mail'],
           user = lambda x: x['Руководитель структурного подразделения'] + ' ' + x['Unnamed: 3'] + ' ' + x['Unnamed: 4'],
            frc = lambda x: x['ЦФО']
           )
    # .iloc[:, 9:]
    [['frc', 'user', 'email', 'login' ]]
), (
    users_raw                    # Ответственные исполнители
    .dropna(subset=['e-mail.1'])
    .assign(login = lambda x: x['e-mail.1'].apply(lambda x: x.split('@')[0]),
            email = lambda x: x['e-mail.1'],
           user = lambda x: x['Ответственный исполнитель '] + ' ' + x['Unnamed: 8'] + ' ' + x['Unnamed: 9'],
            frc = lambda x: x['ЦФО']
           )
    # .iloc[:, 9:]
    [['frc', 'user', 'email', 'login' ]]
)])
users.to_csv("users.csv", index=False)